<a href="https://www.kaggle.com/code/akouaorsot/data-pre-processing-british-airways-reviews?scriptVersionId=140670197" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

### Project 1: British Airways Passengers' Reviews between 2016-2023

With data cleaning taking up a considerable chunk of data science workflow, it is critical to know how to assess, improve, and produce data quality. Every subsequent steps from analysis to machine learning modeling hinges on having good data. And so, this project will focus on practicing best practices when it comes to data cleaning to ensure higher quality and usability. 

In that vein, we examined various sources, especially the following three in an attempt to draft a comprehensive map for the cleaning process: 
* [World Bank Data Cleaning Checklist](https://dimewiki.worldbank.org/Checklist:_Data_Cleaning)
* [Mariann Beargie on Medium](https://mariannbea.medium.com/are-you-sure-your-data-is-completely-clean-use-this-checklist-to-help-8cb415a8b0e0)
* [DataCamp Data Cleaning Infographic](https://www.datacamp.com/blog/infographic-data-cleaning-checklist)

Based on the sources above, we modelled our steps and ensure that we could produce a score on the scale of 1 to 10 before and after cleaning.

# Load libraries & Data

In [1]:
## Importing Libraries
import os           # operating system
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import category_encoders as ce # Encode Categorical Variables

import matplotlib.pyplot as plt # data visualization

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
## Importing the dataset
paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))

paths

['/kaggle/input/british-airways-passenger-reviews-2016-2023/British_Airway_Review.csv']

# Overview

In [3]:
# Snapshot of the first 5 rows
df = pd.read_csv(paths[0])
df.head()

,reviews,date,country,seat_type,recommended,stars,route,type_of_traveller
0,✅ Trip Verified | I had the most fantastic BA...,1st August 2023,Hong Kong,Business Class,yes,5,Heathrow to Las Vegas,Family Leisure
1,✅ Trip Verified | Couldn’t book in online. Ar...,31st July 2023,United Kingdom,Economy Class,no,3,Rome to Heathrow,Solo Leisure
2,✅ Trip Verified | London Heathrow to Mumbai in...,31st July 2023,Iceland,Business Class,yes,3,Gatwick to Venice,Solo Leisure
3,"✅ Trip Verified | Keflavík, Iceland to London ...",31st July 2023,Iceland,Business Class,yes,5,London to Luanda,Couple Leisure
4,✅ Trip Verified | Terrible Experience with Bri...,29th July 2023,Canada,Economy Class,no,5,Denver to Heathrow,Family Leisure


In [4]:
# Overview of the data: 2500 rows, 8 columns, no missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   reviews            2500 non-null   object
 1   date               2500 non-null   object
 2   country            2500 non-null   object
 3   seat_type          2500 non-null   object
 4   recommended        2500 non-null   object
 5   stars              2500 non-null   int64 
 6   route              2500 non-null   object
 7   type_of_traveller  2500 non-null   object
dtypes: int64(1), object(7)
memory usage: 156.4+ KB


In [5]:
# No. of duplicates values detected
df[df.duplicated()]

,reviews,date,country,seat_type,recommended,stars,route,type_of_traveller


In [6]:
# Unique value counts for categorical columns
for col in df.columns[2:]:
    print(df[col].value_counts())

United Kingdom           1552
United States             300
Canada                     80
Australia                  75
Germany                    53
                         ... 
Saint Kitts and Nevis       1
Botswana                    1
Chile                       1
Senegal                     1
Finland                     1
Name: country, Length: 67, dtype: int64
Economy Class      1363
Business Class      754
Premium Economy     248
First Class         135
Name: seat_type, dtype: int64
no     1620
yes     880
Name: recommended, dtype: int64
3    1137
5     682
1     227
9     227
7     227
Name: stars, dtype: int64
Heathrow to Las Vegas    250
Rome to Heathrow         250
Gatwick to Venice        250
London to Luanda         250
Denver to Heathrow       250
BKK to LHR               250
London to Tampa          250
London to Sydney         250
LHR to CPT               250
BLR to LHR               250
Name: route, dtype: int64
Couple Leisure    1000
Family Leisure     500
Solo Leisu

# Feature Transformation/Engineering

In [7]:
# Create a different column "trip verified" to seperate them properly
df['trip_verified'] = np.where(df['reviews'].str.contains("Trip Verified"), 1, 0) 

# Capture the main content of the reviews and create a list of each word 
df['reviews_verif_drop'] = df['reviews'].str.replace("✅ Trip Verified \| ","").str.strip()
df['reviews_bag_of_words'] = df['reviews_verif_drop'].str.split(" ")

# Converting 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# New snapshot of first 5 rows after transformation
df.head()

/tmp/ipykernel_21/2525839514.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['reviews_verif_drop'] = df['reviews'].str.replace("✅ Trip Verified \| ","").str.strip()


,reviews,date,country,seat_type,recommended,stars,route,type_of_traveller,trip_verified,reviews_verif_drop,reviews_bag_of_words
0,✅ Trip Verified | I had the most fantastic BA...,2023-08-01,Hong Kong,Business Class,yes,5,Heathrow to Las Vegas,Family Leisure,1,I had the most fantastic BA Flight today. The ...,"[I, had, the, most, fantastic, BA, Flight, tod..."
1,✅ Trip Verified | Couldn’t book in online. Ar...,2023-07-31,United Kingdom,Economy Class,no,3,Rome to Heathrow,Solo Leisure,1,Couldn’t book in online. Arrived at check in t...,"[Couldn’t, book, in, online., Arrived, at, che..."
2,✅ Trip Verified | London Heathrow to Mumbai in...,2023-07-31,Iceland,Business Class,yes,3,Gatwick to Venice,Solo Leisure,1,London Heathrow to Mumbai in a Boeing 787-8 in...,"[London, Heathrow, to, Mumbai, in, a, Boeing, ..."
3,"✅ Trip Verified | Keflavík, Iceland to London ...",2023-07-31,Iceland,Business Class,yes,5,London to Luanda,Couple Leisure,1,"Keflavík, Iceland to London Heathrow on an A32...","[Keflavík,, Iceland, to, London, Heathrow, on,..."
4,✅ Trip Verified | Terrible Experience with Bri...,2023-07-29,Canada,Economy Class,no,5,Denver to Heathrow,Family Leisure,1,Terrible Experience with British Airways. I bo...,"[Terrible, Experience, with, British, Airways...."


In [8]:

# In the airline industry, seat pricing implies a given order. 
# So, we will encode accordingly
ordinal_encoder = ce.OrdinalEncoder(cols=['seat_type', 'stars'],return_df=True,
                       mapping=[
                           {'col':'seat_type',
                            'mapping':{'Economy Class':0,'Premium Economy':1,
                                            'Business Class':2,'First Class':3}},
                           {'col':'stars',
                            'mapping':{1:0, 3:1, 5:2,7:3, 9:4}}])


# 'recommended' and 'trip_verified' are binary categorical variables so, will be encoded as such
binary_encoder= ce.BinaryEncoder(cols=['recommended', 'trip_verified'],return_df=True)

## 'type_of_traveller' is a nominal categorical variable, 
#So we will one-hot-encoding
one_hot_encoder = ce.OneHotEncoder(cols="type_of_traveller", use_cat_names=True,
                                   return_df=True, )

# The 'country' column is a categorical feature, but with 67 distinct entities
# To avoid increasing the dimensionality in my data set, we
# opted to use frequency encoding.
freq_encoder = ce.CountEncoder(cols="country", normalize=True, return_df=True)

all_encoders = [
    ordinal_encoder,
    binary_encoder,
    one_hot_encoder,
    freq_encoder
]

for enc in all_encoders:
    if enc == freq_encoder:
        df['country_encoded'] = enc.fit_transform(df['country'])
    else:
        df = enc.fit_transform(df)

df.head()

,reviews,date,country,seat_type,recommended_0,recommended_1,stars,route,type_of_traveller_Family Leisure,type_of_traveller_Solo Leisure,type_of_traveller_Couple Leisure,type_of_traveller_Business,trip_verified_0,trip_verified_1,reviews_verif_drop,reviews_bag_of_words,country_encoded
0,✅ Trip Verified | I had the most fantastic BA...,2023-08-01,Hong Kong,2,0,1,2,Heathrow to Las Vegas,1,0,0,0,0,1,I had the most fantastic BA Flight today. The ...,"[I, had, the, most, fantastic, BA, Flight, tod...",0.0052
1,✅ Trip Verified | Couldn’t book in online. Ar...,2023-07-31,United Kingdom,0,1,0,1,Rome to Heathrow,0,1,0,0,0,1,Couldn’t book in online. Arrived at check in t...,"[Couldn’t, book, in, online., Arrived, at, che...",0.6208
2,✅ Trip Verified | London Heathrow to Mumbai in...,2023-07-31,Iceland,2,0,1,1,Gatwick to Venice,0,1,0,0,0,1,London Heathrow to Mumbai in a Boeing 787-8 in...,"[London, Heathrow, to, Mumbai, in, a, Boeing, ...",0.0012
3,"✅ Trip Verified | Keflavík, Iceland to London ...",2023-07-31,Iceland,2,0,1,2,London to Luanda,0,0,1,0,0,1,"Keflavík, Iceland to London Heathrow on an A32...","[Keflavík,, Iceland, to, London, Heathrow, on,...",0.0012
4,✅ Trip Verified | Terrible Experience with Bri...,2023-07-29,Canada,0,1,0,2,Denver to Heathrow,1,0,0,0,0,1,Terrible Experience with British Airways. I bo...,"[Terrible, Experience, with, British, Airways....",0.0320


In [9]:
# Travel routes indicating departures & arrivals
# Unique routes shows that there is an equal distribution of passengers per route
# Additional, there is seems to be an inconsistent formatting of the data points.
# Some routes have airport codes while others don't

# Creating seperate columns for departure & Arrival
locations = df['route'].str.split(" to ").tolist()
df[['Departure','Arrival']] = pd.DataFrame(locations, index= df.index)


# List of airports codes to standardize the information in departure and arrival locations
airports = {
    "London": "LHR",
    "Heathrow": "LHR",
    "Las Vegas": "LAS",
    "Luanda": "LAD",
    "Denver": "DEN",
    "Rome": "FCO",
    "Gatwick": "LGW",
    "Venice": "VCE",
    "Tampa": "TPA",
    "Sydney": "SYD",
    "Bangkok": "BKK",
    "Cape Town": "CPT",
    "Bangalore": "BLR"
}

df.replace({"Departure": airports, 
                "Arrival": airports}, inplace=True)
df.head()

,reviews,date,country,seat_type,recommended_0,recommended_1,stars,route,type_of_traveller_Family Leisure,type_of_traveller_Solo Leisure,type_of_traveller_Couple Leisure,type_of_traveller_Business,trip_verified_0,trip_verified_1,reviews_verif_drop,reviews_bag_of_words,country_encoded,Departure,Arrival
0,✅ Trip Verified | I had the most fantastic BA...,2023-08-01,Hong Kong,2,0,1,2,Heathrow to Las Vegas,1,0,0,0,0,1,I had the most fantastic BA Flight today. The ...,"[I, had, the, most, fantastic, BA, Flight, tod...",0.0052,LHR,LAS
1,✅ Trip Verified | Couldn’t book in online. Ar...,2023-07-31,United Kingdom,0,1,0,1,Rome to Heathrow,0,1,0,0,0,1,Couldn’t book in online. Arrived at check in t...,"[Couldn’t, book, in, online., Arrived, at, che...",0.6208,FCO,LHR
2,✅ Trip Verified | London Heathrow to Mumbai in...,2023-07-31,Iceland,2,0,1,1,Gatwick to Venice,0,1,0,0,0,1,London Heathrow to Mumbai in a Boeing 787-8 in...,"[London, Heathrow, to, Mumbai, in, a, Boeing, ...",0.0012,LGW,VCE
3,"✅ Trip Verified | Keflavík, Iceland to London ...",2023-07-31,Iceland,2,0,1,2,London to Luanda,0,0,1,0,0,1,"Keflavík, Iceland to London Heathrow on an A32...","[Keflavík,, Iceland, to, London, Heathrow, on,...",0.0012,LHR,LAD
4,✅ Trip Verified | Terrible Experience with Bri...,2023-07-29,Canada,0,1,0,2,Denver to Heathrow,1,0,0,0,0,1,Terrible Experience with British Airways. I bo...,"[Terrible, Experience, with, British, Airways....",0.0320,DEN,LHR


# Final Dataset

In [10]:
# After all pre-processing steps,
# Save onto a new csv file
df.to_csv("BA_Flights_Reviews_Pre-processed.csv", header=True)